In [28]:
%load_ext autoreload
%autoreload 2
# Enable imports form top-level of project (edit top_level_path accordingly)
import os
import sys
top_level_path = os.path.abspath(os.path.join('..'))
if top_level_path not in sys.path:
	sys.path.append(top_level_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from pyqubo import Binary, Array, Num
import numpy as np
from dimod import ExactSolver
import neal
from longestpath import gen_average_degree_directed, gen_planted_path, StandardGraph

In [30]:
# graph = StandardGraph(5, [
# 	(0,1),
#     (1,2),
#     (2,3),
#     (3,4)
# ])

graph = gen_average_degree_directed(100, 5)

# max length
N = graph.vertices
M = N

p = M+1

matrix = graph.to_matrix()

In [31]:
vars = Array.create('x', shape=(M+1, N+1), vartype='BINARY')

import time

start = time.time()
serial_exp = Num(-p) * sum([(sum(block) - Num(1))**2 for block in vars]) + Num(0)
end = time.time()

print(f"Done with serial_exp : Took {end - start} seconds")

#Seems to output same matrix as below
# serial_exp = Num(0)

# for m in range(M+1):
# 	temp_exp = Num(-1)
# 	for n in range(N+1):
# 		temp_exp += vars[m][n]
# 	serial_exp += Num(-p) * temp_exp ** 2

no_repeat_blocks_exp = Num(0)

start = time.time()

for i in range(N):
	print(f"{i} / {N-1}")
	for j in range(M):

		no_repeat_blocks_exp += sum([vars[j][i] * vars[k][i] for k in range(j+1, M+1)])

# 		for k in range(j+1, M+1):
# 			no_repeat_blocks_exp += vars[j][i] * vars[k][i] * Num(-p)
no_repeat_blocks_exp *= Num(-p)

end = time.time()

print(f"Done with no_repeatblocks_exp : Took {end - start} seconds")

start = time.time()
edges_exp = Num(0)
for m in range(M):
	for i in range(N):
		for j in range(N):
			if i == j:
				continue
			
			if matrix[i][j]:
				edges_exp += vars[m][i] * vars[m+1][j]
			else:
				edges_exp += Num(-p) * vars[m][i] * vars[m+1][j]

for m in range(M):
	for i in range(N):
		edges_exp += Num(-p) * vars[m][N] * vars[m+1][i]

end = time.time()

print(f"Done with edges_exp : Took {end - start} seconds")

Done with serial_exp : Took 0.01651620864868164 seconds
0 / 99
1 / 99
2 / 99
3 / 99
4 / 99
5 / 99
6 / 99
7 / 99
8 / 99
9 / 99
10 / 99
11 / 99
12 / 99
13 / 99
14 / 99
15 / 99
16 / 99
17 / 99
18 / 99
19 / 99
20 / 99
21 / 99
22 / 99
23 / 99
24 / 99
25 / 99
26 / 99
27 / 99
28 / 99
29 / 99
30 / 99
31 / 99
32 / 99
33 / 99
34 / 99
35 / 99
36 / 99
37 / 99
38 / 99
39 / 99
40 / 99
41 / 99
42 / 99
43 / 99
44 / 99
45 / 99
46 / 99
47 / 99
48 / 99
49 / 99
50 / 99
51 / 99
52 / 99
53 / 99
54 / 99
55 / 99
56 / 99
57 / 99
58 / 99
59 / 99
60 / 99
61 / 99
62 / 99
63 / 99
64 / 99
65 / 99
66 / 99
67 / 99
68 / 99
69 / 99
70 / 99
71 / 99
72 / 99
73 / 99
74 / 99
75 / 99
76 / 99
77 / 99
78 / 99
79 / 99
80 / 99
81 / 99
82 / 99
83 / 99
84 / 99
85 / 99
86 / 99
87 / 99
88 / 99
89 / 99
90 / 99
91 / 99
92 / 99
93 / 99
94 / 99
95 / 99
96 / 99
97 / 99
98 / 99
99 / 99
Done with no_repeatblocks_exp : Took 28.445568561553955 seconds
Done with edges_exp : Took 57.22236919403076 seconds


In [32]:
def to_matrix(exp):
	model = exp.compile()
	qubo, energy_offset = model.to_qubo(index_label=True)

	return np.matrix([[int(qubo.get((i,j), 0)) for j in range((M+1)*(N+1))] for i in range((M+1)*(N+1))])

total_exp = serial_exp + no_repeat_blocks_exp + edges_exp

exp = -total_exp

model = exp.compile()
bqm = model.to_bqm()

# print(to_matrix(total_exp))

start = time.time()

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.energy)

end = time.time()

print(f"Solving took {end - start} seconds")

# sampleset = ExactSolver().sample(bqm)
# decoded_samples = model.decode_sampleset(sampleset)
# best_sample = max(decoded_samples, key=lambda s: s.energy)
# print(best_sample.energy)